In [1]:

import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import math

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import f1_score


In [2]:
DATA_DIR = pathlib.Path("Z:\\Dropbox\\www\\studs.steelfeet.ru\\_hack\\2021-22\global-ai")
#DATA_DIR = pathlib.Path("C:\\0")
DATA_FILE = "train.csv"
TEST_FILE = "test.csv"
#group_by столбцы
AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]


# Загрузка данных

In [3]:
import random
data =[]
active = 0
no_active = 0
with open(DATA_DIR.joinpath(DATA_FILE), 'r') as file_name:
    reader = csv.DictReader(file_name)
    for row in reader:
        if (row["Active"] == "True"):
            active += 1
            data.append((row["Smiles"], row["Active"]))
        else:

            # подгоняем статистику, чтобы и тех и других было поровну
            if (active > no_active):
                if (random.random()>0.5):
                    no_active += 1
                    data.append((row["Smiles"], row["Active"]))


print(len(data), active, active / len(data))
data
# df = pd.read_csv(DATA_DIR.joinpath(DATA_FILE), usecols=["Smiles", "Active"])
# df["Active"]



412 206 0.5


[('Nc1ccc(O)c2ncccc12', 'True'),
 ('Cc1noc(-c2ccc(-c3ccc(CC(=O)O)cc3)cc2)c1NC(=O)O[C@H](C)c1ccccc1Cl', 'False'),
 ('C[C@H]1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23', 'True'),
 ('C[C@H](NC(=O)C(C)(F)F)[C@H](Oc1ccc2c(cnn2-c2cccc(C(=O)N[C@@H]3CCOC3)c2)c1)c1ccc2c(c1)OCCO2',
  'False'),
 ('Cl.NCCc1cc(O)c(O)cc1O', 'True'),
 ('O=C(O)Cn1nnc(-c2cc(N3CCC(Oc4cc(F)ccc4Br)CC3)no2)n1', 'False'),
 ('Cc1ccc(-n2sc(=O)n(Cc3ccc(F)cc3)c2=O)cc1', 'True'),
 ('COCCN1CCN(Cc2ccc(-c3n[nH]c4c3C(=O)c3c(NC(=O)NN5CCOCC5)cccc3-4)cc2)CC1',
  'False'),
 ('Oc1cc2c(cc1C(c1ccc(C(F)(F)F)cc1)N1CCOCC1)OCO2', 'True'),
 ('CCOC(=O)C1CN(Cc2ccccc2)C(=O)C1=O', 'False'),
 ('CN1CCC(CNc2ccc3ncc(-c4cccc(OC(F)(F)F)c4)n3n2)CC1', 'True'),
 ('c1ccc(CNc2ncnc3[nH]cnc23)cc1', 'False'),
 ('CC(Cc1ccc(O)c(O)c1)C(C)Cc1ccc(O)c(O)c1', 'True'),
 ('O=C(NCC(=O)N1CCCC1)Nc1ccc2cnccc2c1', 'True'),
 ('OCc1cncc(F)c1', 'False'),
 ('COc1cccc(OC)c1-c1cc(C(=O)N[C@@H](CC(C)C)C(=O)O)nn1-c1ccnc2cc(Cl)ccc12',
  'False'),
 ('CC[C@@H](C)Nc1cc(C(=O)NC2CC3CCC(C

# Целевая переменная (labels)

In [4]:
y = []
for line in data:
    smile, active = line
    if (str(active) == "False"):
        y.append(0)
    else:
        y.append(1)




In [5]:
#https://vc.ru/newtechaudit/340387-predskazanie-kolichestva-laykov-u-kommentariya-v-youtube
def process_text(text):
    process_text = ""
    line = text
    for n in range(0, len(line)):
        s1 = str(line[n])
        process_text = process_text + " " + s1
        try:
            s2 = str(line[n]+line[n+1])
            process_text = process_text + " " + s2
        except:
            pass
        try:
            s3 = str(line[n]+line[n+1]+line[n+2])
            process_text = process_text + " " + s3
        except:
            pass
        try:
            s4 = str(line[n]+line[n+1]+line[n+2]+line[n+3])
            process_text = process_text + " " + s4
        except:
            pass

        try:
            s5 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4])
            process_text = process_text + " " + s5
        except:
            pass

        try:
            s6 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5])
            process_text = process_text + " " + s6
        except:
            pass

        try:
            s7 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6])
            process_text = process_text + " " + s7
        except:
            pass
        try:
            s8 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6]+line[n+7])
            process_text = process_text + " " + s8
        except:
            pass
        
        """
        try:
            s9 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6]+line[n+7]+line[n+8])
            process_text = process_text + " " + s9
        except:
            pass

        try:
            s10 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6]+line[n+7]+line[n+8]+line[n+9])
            process_text = process_text + " " + s10
        except:
            pass
        """


    return process_text

X = []
for line in data:
    smile, active = line
    X.append(process_text(smile))



In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Векторизация обработанных слов
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


vectorizer_vocabulary = vectorizer.vocabulary_
#feature_importances
#print(vectorizer.vocabulary_)
vectorizer_vocabulary

{'nc': 1987,
 'nc1': 1988,
 'nc1c': 1990,
 'nc1c2': 1991,
 'nc1c2c': 1992,
 'nc1c2cc': 1993,
 'nc1c2ccc': 1994,
 'c1': 610,
 'c1c': 615,
 'c1c2': 620,
 'c1c2c': 621,
 'c1c2cc': 622,
 'c1c2ccc': 623,
 'c1c2cccc': 624,
 '1c': 10,
 '1c2': 17,
 '1c2c': 19,
 '1c2cc': 20,
 '1c2ccc': 21,
 '1c2cccc': 22,
 '1c2ccccc': 23,
 'c2': 738,
 'c2c': 747,
 'c2cc': 771,
 'c2ccc': 786,
 'c2cccc': 796,
 'c2ccccc': 800,
 'c2ccccc2': 802,
 '2c': 204,
 '2cc': 238,
 '2ccc': 254,
 '2cccc': 267,
 '2ccccc': 273,
 '2ccccc2': 276,
 '2ccccc2n': 279,
 'cc': 1051,
 'ccc': 1241,
 'cccc': 1341,
 'ccccc': 1397,
 'ccccc2': 1408,
 'ccccc2n': 1416,
 'ccccc2nc': 1418,
 'cccc2': 1356,
 'cccc2n': 1368,
 'cccc2nc': 1370,
 'cccc2nc2': 1371,
 'ccc2': 1261,
 'ccc2n': 1287,
 'ccc2nc': 1289,
 'ccc2nc2': 1290,
 'ccc2nc2c': 1291,
 'cc2': 1103,
 'cc2n': 1149,
 'cc2nc': 1151,
 'cc2nc2': 1152,
 'cc2nc2c': 1153,
 'cc2nc2cc': 1154,
 'c2n': 831,
 'c2nc': 833,
 'c2nc2': 834,
 'c2nc2c': 835,
 'c2nc2cc': 836,
 'c2nc2ccc': 837,
 '2n': 327,
 '2n

In [7]:
cat_features = list(range(0, X_train.shape[1]))
#cat_features.append("Active")
len(cat_features)

2399

In [8]:
catboost = CatBoostRegressor(loss_function='RMSE', random_seed=0, silent=True, cat_features=cat_features)
catboost.fit(X_train, y_train, plot=True)
#y_pred = catboost.predict(X_test)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# Отбираем только важные слова

In [9]:
feature_importances = pd.Series(catboost.feature_importances_).sort_values(ascending=False).to_dict()
feature_importances



{1052: 8.103520270970677,
 1341: 4.4132413009001015,
 631: 4.389058282757818,
 747: 2.766753277783599,
 2198: 2.3931102154947363,
 1987: 2.2744634424478907,
 1871: 1.9709501478542897,
 1051: 1.7937877078537392,
 727: 1.5955763515247796,
 1784: 1.5705898102469107,
 1528: 1.539126539544186,
 738: 1.5262310986799403,
 10: 1.240463441983897,
 1107: 1.1659853418934931,
 2339: 1.1653223093847738,
 238: 1.128661611344693,
 184: 1.1031058982358573,
 1641: 1.0782924903177362,
 2077: 1.0512710333469388,
 648: 1.0453121491611275,
 204: 1.0184169977921584,
 56: 0.9851535109941194,
 895: 0.9592107593461093,
 615: 0.9410984267993643,
 1062: 0.9301585408779195,
 1057: 0.9048535165337571,
 1640: 0.8360995082102866,
 771: 0.8199356880981001,
 1642: 0.7752138861108957,
 31: 0.7704346966332007,
 881: 0.7105338915784398,
 327: 0.7101131107456914,
 148: 0.7010023925600976,
 2195: 0.6526458192431418,
 2025: 0.6454081498175027,
 1633: 0.6446901902807265,
 1397: 0.632539036769186,
 403: 0.6089297670871736,
 3

In [10]:
"""
vectorizer_vocabulary
{'nc': 2544,
 'nc1': 2545,
"""
# меняем местами ключи и значения
vocabulary_vectorizer = dict(zip(vectorizer_vocabulary.values(), vectorizer_vocabulary.keys()))

non_potent_words = []
for id, potent in feature_importances.items():
    if (potent < 0.1):
        try:
            non_potent_words.append(vocabulary_vectorizer[id])
        except:
            pass
    
print(len(non_potent_words), len(vectorizer_vocabulary))

2240 2399


# Перезагружаем ВЕСЬ датасет, добавляя случайные незначащие слова

In [11]:
data_active = []
data_no_active = []
with open(DATA_DIR.joinpath(DATA_FILE), 'r') as file_name:
    reader = csv.DictReader(file_name)
    for row in reader:
        if (row["Active"] == "True"):
            data_active.append((row["Smiles"], row["Active"]))
        else:
            data_no_active.append((row["Smiles"], row["Active"]))


print(len(data_active), len(data_no_active), len(data_active) / len(data_no_active))

206 5351 0.03849747710708279


In [12]:
# добавляем незначащие слова к Smile
# очень сильно увеличивает число новых слов => нет, столько-же
for n in range(0, (len(data_no_active) - len(data_active))):
    random_word_len = int(random.random() * 10 + 3)
    random_word = ""
    for i in range(0, random_word_len):
        random_word = random_word + random.choice(non_potent_words)
    
    new_data = random.choice(data_active)[0] + random_word, "True"
    data_active.append(new_data)

print(len(data_active), len(data_no_active), len(data_active) / len(data_no_active)) 

data = data_active + data_no_active

5351 5351 1.0


In [22]:
# добавляем незначащие слова к массиву слов
def process_text_active(text):
    process_text = ""
    line = text
    for n in range(0, len(line)):
        s1 = str(line[n])
        process_text = process_text + " " + s1
        try:
            s2 = str(line[n]+line[n+1])
            process_text = process_text + " " + s2
        except:
            pass
        try:
            s3 = str(line[n]+line[n+1]+line[n+2])
            process_text = process_text + " " + s3
        except:
            pass
        try:
            s4 = str(line[n]+line[n+1]+line[n+2]+line[n+3])
            process_text = process_text + " " + s4
        except:
            pass

        try:
            s5 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4])
            process_text = process_text + " " + s5
        except:
            pass

        try:
            s6 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5])
            process_text = process_text + " " + s6
        except:
            pass

        try:
            s7 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6])
            process_text = process_text + " " + s7
        except:
            pass
        try:
            s8 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6]+line[n+7])
            process_text = process_text + " " + s8
        except:
            pass

        """
        try:
            s9 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6]+line[n+7]+line[n+8])
            process_text = process_text + " " + s9
        except:
            pass

        try:
            s10 = str(line[n]+line[n+1]+line[n+2]+line[n+3]+line[n+4]+line[n+5]+line[n+6]+line[n+7]+line[n+8]+line[n+9])
            process_text = process_text + " " + s10
        except:
            pass
        """
    random_word_len = int(random.random() * 5 + 2)
    for i in range(0, random_word_len):
        process_text = process_text + " " + random.choice(non_potent_words)

    return process_text

In [23]:
y = []
X = []

for line in data_active:
    smile, active = line
    y.append(1)
    X.append(process_text_active(smile))

for line in data_no_active:
    smile, active = line
    y.append(0)
    X.append(process_text(smile))


print(len(X), len(y))

10702 10702


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=0)

# Векторизация обработанных слов
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


vectorizer_vocabulary = vectorizer.vocabulary_
#vectorizer_vocabulary

cat_features = list(range(0, X_train.shape[1]))
#cat_features.append("Active")
len(cat_features)

71007

In [38]:
catboost = CatBoostRegressor(loss_function='RMSE', random_seed=0, silent=True, cat_features=cat_features)
catboost.fit(X_train, y_train, plot=True)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: bad allocation

In [92]:
# проверка на Train
y_pred_train = catboost.predict(X_train)
n = 0
mean_true = 0
max_true = 0
min_true = 10

mean_false = 0
max_false = 0
min_false = 10


n_true = 0
n_false = 0
for y_pred in y_pred_train:
    y_real = y_train[n]
    if (y_real == 0):
        mean_false = mean_false + y_pred
        if (y_pred > max_false):
            max_false = y_pred
        if (y_pred < min_false):
            min_false = y_pred
        n_false += 1
    else:
        mean_true = mean_true + y_pred
        if (y_pred > max_true):
            max_true = y_pred
        if (y_pred < min_false):
            min_true = y_pred
        n_true += 1

    n+=1

mean_false = mean_false / n_false
mean_true = mean_true / n_true
print(min_false, mean_false, max_false, " | ", min_true, mean_true, max_true)


-0.06579868845769143 0.28299823439869637 0.5950958741008743  |  0.5785066995018072 0.6009758341971256 1.0256120698828066


In [93]:
# проверка на Val
y_pred_test = catboost.predict(X_test)

y_pred_cat = []
for y_pred in y_pred_test:
    if (y_pred < 0.557):
      y_pred_cat.append(0)
    else:
      y_pred_cat.append(1)


print(f1_score(y_test, y_pred_cat))


0.3505154639175258


In [79]:
test_data =[]
with open(DATA_DIR.joinpath(TEST_FILE), 'r') as file_name:
    reader = csv.DictReader(file_name)
    for row in reader:
        test_data.append((row["Smiles"]))


print(len(test_data))

1614


In [80]:
X_test = []
for line in test_data:
    X_test.append(process_text(line))

X_test = vectorizer.transform(X_test)

X_test

<1614x2786 sparse matrix of type '<class 'numpy.int64'>'
	with 65035 stored elements in Compressed Sparse Row format>

In [81]:
y_pred_test = catboost.predict(X_test)

y_pred_cat = []
text = ",Active\n"
n = 1
for y_pred in y_pred_test:
    if (y_pred < 0.557):
      y_pred_cat.append(0)
      text = text + str(n) + ",False\n"
    else:
      y_pred_cat.append(1)
      text = text + str(n) + ",True\n"

file = open('submission.csv', 'w')
file.write(text)
file.close()
